# Setup

In [ ]:
# import packages
import pandas as pd

In [ ]:
# load data from raw dir
df_game_reviews = pd.read_csv(r'../data/raw/game_reviews_raw.csv', low_memory=False)

In [ ]:
#subset english-only revies
df_game_reviews_english = df_game_reviews[df_game_reviews['language']=='english']

#sent review to a list 
reviews = df_game_reviews_english['review'].tolist()

In [ ]:
reviews = reviews[:10]

# 1. Apply K-Means to data you have collected and cleaned with k=5.

In [ ]:
from bert_serving.client import BertClient
from sklearn.cluster import KMeans

In [ ]:
bc = BertClient()
BERT_embedding = bc.encode(reviews)

km = KMeans(n_clusters=5)
km.fit(BERT_corpus)
clusters = km.labels_.tolist()

In [ ]:
from bert_serving.client import BertClient
bc = BertClient()
bc.encode(['First do it', 'then do it right', 'then do it better'])

In [ ]:
import torch
import numpy as np
from kmeans_pytorch import kmeans

# data
data_size, dims, num_clusters = 1000, 2, 3
x = np.random.randn(data_size, dims) / 6
x = torch.from_numpy(x)

# kmeans
cluster_ids_x, cluster_centers = kmeans(
    X=x, num_clusters=num_clusters, distance='euclidean', device=torch.device('cuda:0')
)

In [ ]:

import torch
import numpy as np
import matplotlib.pyplot as plt
from kmeans_pytorch import kmeans, kmeans_predict

In [ ]:
# set random seed
np.random.seed(123)

In [ ]:

# data
data_size, dims, num_clusters = 1000, 2, 3
x = np.random.randn(data_size, dims) / 6
x = torch.from_numpy(x)

In [ ]:
# set device
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [ ]:
# k-means
cluster_ids_x, cluster_centers = kmeans(X=x, 
                                        num_clusters=num_clusters, 
                                        distance='euclidean', 
                                        device=device)

In [ ]:
print(cluster_ids_x)

In [ ]:
2. Use silhouette or elbow method to find k and recluster your data.

3. Compare the clustering results from both. Do the clusters make sense? Which performed better? 

In [ ]:
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings

In [ ]:
# initialize the word embeddings
glove_embedding = WordEmbeddings('glove')

In [ ]:
from flair.data import Sentence
from flair.embeddings import SentenceTransformerDocumentEmbeddings

# init embedding
embedding = SentenceTransformerDocumentEmbeddings('bert-base-nli-mean-tokens')

# create a sentence
sentence = Sentence('The grass is green .')

# embed the sentence
embedding.embed(sentence)

In [ ]:
from flair.embeddings import TransformerDocumentEmbeddings

# init embedding
embedding = TransformerDocumentEmbeddings('bert-base-uncased')

# create a sentence
sentence = Sentence('The grass is green .')

# embed the sentence
embedding.embed(sentence)